### **bioalerts: A Python Package for the Derivation of Structural Alerts from Bioactivity Data Sets**


Isidro Cortés Ciriano (isidrolauscher@gmail.com). 2015-2016.



This notebook serves to compare the **bioalerts** implementation for the derivation of structural alerts from categorical data to the original paper by Ahlberg et al. (J. Chem. Inf. Model., 2014, 54 (10), pp 2945–2952).

The only differences between both methods are related to how the substructures are generated. Bioalerts computes substructures using Morgan fingerprints as implemented in the rdkit (rdkit.org). See the main text for further details.

Here, we use the same training and test data sets used in Ahlberg et al. 

Firstly, import **bioalerts** and other packages required for this tutorial.


Note: do not forget to add the path to your python envirnonment variable, e.g.:
export PYTHONPATH=$PYTHONPATH:/usr/local/share/RDKit:/Users/user1/Desktop/bioalerts/

In [1]:
import bioalerts
import numpy as np

In [2]:
from bioalerts import LoadMolecules, Alerts, FPCalculator

In [3]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools
import time

This notebook was run with the following python and module versions:

In [4]:
import sys, numpy as np, scipy as sc, rdkit, matplotlib as pylab, pandas as pd, IPython
print " Python:", sys.version, "\n"
print " Numpy:", np.__version__
print " Scipy:", sc.__version__
print " Rdkit:", rdkit.rdBase.rdkitVersion
print " Matplotlib:", pylab.__version__
print " Pandas:", pd.__version__
print " Ipython:", IPython.__version__

 Python: 2.7.2 (default, Oct 11 2012, 20:14:37) 
[GCC 4.2.1 Compatible Apple Clang 4.0 (tags/Apple/clang-418.0.60)] 

 Numpy: 1.9.2
 Scipy: 0.16.0
 Rdkit: 2014.09.2
 Matplotlib: 1.4.3
 Pandas: 0.15.2
 Ipython: 3.0.0


In [5]:
training = bioalerts.LoadMolecules.LoadMolecules("./datasets/training_Ahlberg.smi",delimiter="\t",name_field=None)
                                               
training.ReadMolecules(titleLine=False,smilesColumn=0,nameColumn=1)
print "Total number of input molecules: ", len(training.mols)

Format of the structures file = SMILES
All molecules in the input file were processed correctly
Total number of input molecules:  4254


In [6]:
test = bioalerts.LoadMolecules.LoadMolecules("./datasets/test_Ahlberg.smi",delimiter="\t",name_field=None)
test.ReadMolecules(titleLine=True)
print "Total number of input molecules: ", len(test.mols)

Format of the structures file = SMILES
2 molecules (starting at zero) could not be processed.

This information has been saved in the following file: incorrect_molecules.csv

NOTE: the indexes of the molecules start at zero. Thus the first molecule is molecule 0.
Total number of input molecules:  830


In this particular case, the second column of the smiles file contains the labels (positive or negative) instead of the molecule names, which are generally in the column. This is the reason why the variable **mol_ids** contains the labels.

Load the labels for the training and test sets

In [7]:
labels_training = np.genfromtxt('./datasets/training_Ahlberg_activities.txt',
                              dtype='str',
                              skip_header=0,
                              usecols=0)
arr = np.arange(0,len(labels_training))
mask = np.ones(arr.shape,dtype=bool)
mask[training.molserr]=0
labels_training =  labels_training[mask]

In [8]:
labels_test = np.genfromtxt('./datasets/test_Ahlberg_activities.txt',
                              dtype='str',
                              skip_header=0,
                              usecols=0)
arr = np.arange(0,len(labels_test))
mask = np.ones(arr.shape,dtype=bool)
mask[test.molserr]=0
labels_test =  labels_test[mask]

In [9]:
print len(labels_training), len(labels_test)

4254 830


Next, we are going to extract the substructure information from the molecules contained in the training set.

In [10]:
training_set_info = bioalerts.LoadMolecules.GetDataSetInfo(name_field=None)

In [11]:
initial_time = time.clock()
training_set_info.extract_substructure_information(radii=[1,2,3,4],mols=training.mols)
print round(time.clock() - initial_time)/60, " minutes"

2.48333333333  minutes


In [12]:
initial_time = time.clock()
# the maximum radius corresponds to the maximum value in the list radii used above (i.e. [2,3,4])
Alerts_categorical_70 = bioalerts.Alerts.CalculatePvaluesCategorical(max_radius=4)


Alerts_categorical_70.calculate_p_values(mols=test.mols,
                                      substructure_dictionary=training_set_info.substructure_dictionary,
                                      bioactivities=labels_training,
                                      mols_ids=training.mols_ids,
                                      threshold_nb_substructures = 5,
                                      threshold_pvalue = 0.05,
                                      threshold_frequency = 0.7,
                                      Bonferroni=False,
                                      active_label='POS',
                                      inactive_label='NEG')
print round(time.clock() - initial_time)/60, " minutes"

Number of substructures processed:  37035
Significant substructures:  1025 substructures
0.816666666667  minutes


In [15]:
initial_time = time.clock()
Alerts_categorical_80 = bioalerts.Alerts.CalculatePvaluesCategorical(max_radius=4)


Alerts_categorical_80.calculate_p_values(mols=test.mols,
                                      substructure_dictionary=training_set_info.substructure_dictionary,
                                      bioactivities=labels_training,
                                      mols_ids=training.mols_ids,
                                      threshold_nb_substructures = 5,
                                      threshold_pvalue = 0.05,
                                      threshold_frequency = 0.8,
                                      Bonferroni=False,
                                      active_label='POS',
                                      inactive_label='NEG')
print round(time.clock() - initial_time)/60, " minutes"

Number of substructures processed:  37035
Significant substructures:  823 substructures
0.883333333333  minutes


In [16]:
initial_time = time.clock()
Alerts_categorical_80_Bonferroni_True = bioalerts.Alerts.CalculatePvaluesCategorical(max_radius=4)


Alerts_categorical_80_Bonferroni_True.calculate_p_values(mols=test.mols,
                                      substructure_dictionary=training_set_info.substructure_dictionary,
                                      bioactivities=labels_training,
                                      mols_ids=training.mols_ids,
                                      threshold_nb_substructures = 5,
                                      threshold_pvalue = 0.05,
                                      threshold_frequency = 0.8,
                                      Bonferroni=True,
                                      active_label='POS',
                                      inactive_label='NEG')
print round(time.clock() - initial_time)/60, " minutes"

Number of substructures processed:  37035
Significant substructures:  526 substructures
0.8  minutes


In Ahlberg et al. JCIM 2014, it was stated that:  

"For the 70% case, 17600 signatures were analyzed and 985 of those found significant, and for the 80% case, 24300
signatures were evaluated and 1137 found significant. When applying the Benjamini Hochberg procedure, the number of significant signatures was reduced to 133."

Here we find (when considering substructures of radius 1, 2, 3, 4): 

In [17]:
print len(Alerts_categorical_70.output) # Bonferroni False; 
print len(Alerts_categorical_80.output) # Bonferroni False
print len(Alerts_categorical_80_Bonferroni_True.output) # Bonferroni False

1025
823
526


Overall, these numbers are comparable considering the differences with respect to the generation of compound substructures. 

                                
                                
                                              **END**
                                
         For any doubts or suggestions, please contact me at isidrolauscher [] gmail [] com